# 分析正负先导的发展特征

## 1.加载三维的结果

In [198]:
# 加载对应的python库
import pandas as pd
import numpy as np
three_d_result = pd.read_csv("../two_station_3D/3d_win512_cost_cal_yld_chj_dtoa.csv")
filtered_3d_result = three_d_result[
    (three_d_result.dlta < 20000) & 
    (three_d_result.yld_start_loc > 3.88e8) & 
    (three_d_result.yld_start_loc < 4e8) & 
    (three_d_result.r_gccs > 0.1) & 
    (abs(three_d_result.R3_value) < 10000) & 
    (three_d_result.x < 6000) & 
    (three_d_result.x > -10000) & 
    (three_d_result.y < 0) & 
    (three_d_result.y > -10000) & 
    (three_d_result.z < 9000) & 
    (three_d_result.z > 500)
]
filtered_3d_result

,yld_start_loc,chj_loc,chj_azimuth,chj_elevation,r_gccs,dlta,R3_value,S_initial_triangulation_1,S_initial_triangulation_2,S_initial_triangulation_3,chi_square_red,cost,dlta_T,x,y,z
2059,388103901,422324360,334.725749,45.021626,0.430682,2558.119840,370.597600,-116.092315,-2804.029001,5509.252613,7.618947,0.455454,-2774.644527,-106.092799,-2814.029001,5519.252613
2060,388142974,422362173,344.141903,40.124320,0.338449,10822.299729,-439.762609,114.184331,-2322.258222,4834.217575,118.967028,0.529241,3377.857170,124.184331,-2332.258222,4844.217575
2061,388153486,422374625,346.077746,43.801359,0.285643,2621.109168,531.236009,771.666843,-1320.040508,6275.894457,8.995128,0.571486,-6370.714720,774.832808,-1330.040508,6285.894457
2063,388284846,422507950,334.433987,45.571859,0.243938,11434.116043,339.309123,-167.504458,-2810.462320,5628.407240,130.662553,0.604850,-16715.779548,-177.504458,-2800.462320,5618.407240
2064,388285025,422505210,333.745058,44.604103,0.262699,3426.611934,425.327419,-198.720084,-2767.154076,5510.163999,13.169743,0.589841,-2110.091547,-188.720115,-2777.154076,5520.163999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4823,399938906,434146257,61.207608,30.971788,0.354787,1307.121340,-328.771762,4607.068203,-6405.771037,1807.513188,2.023199,0.516170,16651.327188,4597.068203,-6415.771037,1797.513188
4824,399939049,434146257,61.207608,30.971788,0.345221,2022.121317,-328.771783,4607.068195,-6405.771040,1807.513186,4.330871,0.523824,17366.327188,4597.068195,-6415.771029,1797.513186
4825,399940685,434146546,19.848992,51.837922,0.381024,12677.806637,1891.055638,2974.126505,-5691.591063,2522.002928,198.999439,0.495181,24100.199877,2984.126470,-5701.591063,2512.002928
4826,399945436,434150789,61.207037,31.277565,0.382344,9923.670331,-44.637233,4248.267105,-6610.530163,1534.297095,97.412650,0.494124,26623.649080,4238.267105,-6620.530163,1524.297095


## 2. 处理三维数据，根据结果的高度，对闪电进行分段

In [199]:
filtered_height_3d_result = filtered_3d_result[
    (filtered_3d_result.z > 1000 ) & (filtered_3d_result.z < 2500 ) &
    (filtered_3d_result.yld_start_loc > 3.989e8 ) & (filtered_3d_result.yld_start_loc < 4e8 )
]
filtered_height_3d_result.yld_start_loc.describe()
# filtered_height_3d_result.yld_start_loc

count    2.010000e+02
mean     3.994205e+08
std      2.917037e+05
min      3.989005e+08
25%      3.991582e+08
50%      3.994549e+08
75%      3.996711e+08
max      3.999986e+08
Name: yld_start_loc, dtype: float64

In [200]:
filtered_height_3d_result = filtered_3d_result[
    (filtered_3d_result.z > 2500 ) & (filtered_3d_result.z < 4000 ) &
    (filtered_3d_result.yld_start_loc > 3.965e8 ) & (filtered_3d_result.yld_start_loc < 3.98e8 )
]
filtered_height_3d_result.yld_start_loc.describe()

count    3.510000e+02
mean     3.974714e+08
std      3.675411e+05
min      3.965024e+08
25%      3.972307e+08
50%      3.975057e+08
75%      3.977837e+08
max      3.979969e+08
Name: yld_start_loc, dtype: float64

## 3. 统计梯级个数(基于二维结果)

In [201]:
# 加载2d定位结果数据
result = pd.read_csv('result_yld_3.8e8_4e8_window_512_128_去零飘_滤波_加窗_阈值15_30_80.txt', sep=r'\s+')

# 对2d结果进行过滤筛选
filtered_result = result[
    (abs(result['t123']) < 1) &  # t123的绝对值小于1
    (abs(result['Rcorr']) > 0.3) &  # Rcorr的绝对值大于0.65
    (result['Start_loc'] < 4e8) &  # Start_loc小于4e8
    (result['Start_loc'] > 3.8e8)  
]
filtered_result

,Start_loc,peak,t12,t13,t23,cos_alpha_opt,cos_beta_opt,Azimuth,Elevation,Rcorr,t123
2,380003414,256,35.1,-7.0,-42.5,-0.217557,-0.617881,199.397285,49.075565,0.513815,-0.4
3,380003653,256,34.6,-7.0,-41.9,-0.213945,-0.609365,199.345903,49.771928,0.669812,-0.3
4,380003828,256,35.2,-7.1,-42.6,-0.217742,-0.619647,199.361297,48.944474,0.689729,-0.3
5,380004006,256,35.3,-7.1,-42.6,-0.218353,-0.620290,199.393000,48.882896,0.700022,-0.2
6,380004243,256,35.4,-7.4,-43.3,-0.218052,-0.627832,199.152571,48.346799,0.640584,-0.5
...,...,...,...,...,...,...,...,...,...,...,...
50910,399997030,256,-14.1,-94.5,-80.5,0.503080,-0.846286,149.270364,10.092977,0.498249,-0.1
50914,399998226,256,-2.2,-82.2,-79.7,0.367100,-0.886351,157.502151,16.389784,0.442881,0.3
50915,399998465,256,-26.0,-101.3,-76.1,0.615276,-0.747139,140.528244,14.562296,0.591258,-0.8
50916,399998596,256,-25.8,-101.4,-75.7,0.614941,-0.745134,140.467990,14.958218,0.675269,-0.1


## 4. 计算负先导的梯级个数、梯级间隔、梯级步长

### 5500-7000高度

In [302]:
# 选出负先导数据
start_loc = 3.9e8
end_loc = 3.94e8
negative_lead_result = filtered_result[
    (filtered_result.Start_loc >= start_loc) &
    (filtered_result.Start_loc < end_loc)
]
time_diffs = np.diff(negative_lead_result.Start_loc)
max_diff_idx = np.argmax(time_diffs)
corresponding_start_loc = negative_lead_result.Start_loc.iloc[max_diff_idx]
print("最大的间隔位于:",corresponding_start_loc)

# 查看时间间隔大于某阈值的个数

thea = 7000
filtered_negative_time_diffs = time_diffs[ time_diffs > thea]
print(f"时间间隔大于{thea}的个数有：{len(filtered_negative_time_diffs)}")
print(f"负先导 {start_loc}-{end_loc} 阶段的时间间隔分布情况：\n",pd.DataFrame(data=filtered_negative_time_diffs).describe())

最大的间隔位于: 392790685
时间间隔大于7000的个数有：86
负先导 390000000.0-394000000.0 阶段的时间间隔分布情况：
                   0
count     86.000000
mean    9086.883721
std     2221.406340
min     7001.000000
25%     7526.750000
50%     8752.500000
75%     9491.750000
max    19116.000000


In [303]:
from sklearn.cluster import DBSCAN
from scipy import stats
# 计算梯级步长
## Step1: 拿到每个间隔对应的位置范围
filtered_indices = np.where(time_diffs > thea)
filtered_start_loc = negative_lead_result.Start_loc.iloc[filtered_indices[0]].values    # .values 将结果转为Numpy数组
start_locs = filtered_start_loc[:-1]  # 去掉最后一个元素
end_locs = filtered_start_loc[1:]    # 去掉第一个元素
# 拼接为元组列表
loc_tuples = list(zip(start_locs, end_locs))

# --- DBSCAN 聚类参数 ---
epsilon = 300  
min_samples = 3

distances = []
# Step2: 三维结果中筛选出对应的结果
for i, loc_tuple in enumerate(loc_tuples):
    # 按时间范围筛选结果
    filtered_loc_3d_result = filtered_3d_result[
        (filtered_3d_result.yld_start_loc > loc_tuple[0] - 1) &
        (filtered_3d_result.yld_start_loc < loc_tuple[1] + 1)
    ]
    if len(filtered_loc_3d_result) > min_samples:
        
        # ====================DBSCAN聚类去噪====================
        points_in_window = filtered_loc_3d_result[['x', 'y', 'z']].values
        db = DBSCAN(eps=epsilon, min_samples=min_samples).fit(points_in_window)
        cluster_labels = db.labels_
        core_samples_labels = cluster_labels[cluster_labels != -1]
        # 找到出现次数最多的簇ID
        main_cluster_id = stats.mode(core_samples_labels).mode
        main_path_mask = (cluster_labels == main_cluster_id)
        if np.sum(main_path_mask) < 2:
            continue
        # 使用掩码从DataFrame中获取主路径的所有数据
        main_path_df = filtered_loc_3d_result[main_path_mask]
        # 对筛选出的主路径点，按时间重新排序
        main_path_sorted_df = main_path_df.sort_values(by='yld_start_loc')
        # 获取并输出主路径的起止位置
        #    .iloc[0] 获取第一行, .iloc[-1] 获取最后一行
        start_position = main_path_sorted_df.iloc[0][['x','y','z']].values
        end_position = main_path_sorted_df.iloc[-1][['x','y','z']].values
        
# Step3: 计算三维距离（步长）
        distance = np.linalg.norm(end_position-start_position)
        distances.append(distance)
    

distances_df = pd.DataFrame(data=distances)
print(f"负先导 {start_loc}-{end_loc} 阶段的梯级步长分布情况：\n",distances_df.describe())
        

C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample 

负先导 390000000.0-394000000.0 阶段的梯级步长分布情况：
                 0
count   31.000000
mean   119.137122
std     66.624474
min     42.118675
25%     83.754952
50%     93.173294
75%    133.315726
max    289.458478


### 4000-5000高度

In [304]:
# 选出负先导数据
start_loc = 3.94e8
end_loc = 3.965e8
negative_lead_result = filtered_result[
    (filtered_result.Start_loc >= start_loc) &
    (filtered_result.Start_loc < end_loc)
]
time_diffs = np.diff(negative_lead_result.Start_loc)
max_diff_idx = np.argmax(time_diffs)
corresponding_start_loc = negative_lead_result.Start_loc.iloc[max_diff_idx]
print("最大的间隔位于:",corresponding_start_loc)

# 查看时间间隔大于某阈值的个数

thea = 6000
filtered_negative_time_diffs = time_diffs[ time_diffs > thea]
print(f"时间间隔大于{thea}的个数有：{len(filtered_negative_time_diffs)}")
print(f"负先导 {start_loc}-{end_loc} 阶段的时间间隔分布情况：\n",pd.DataFrame(data=filtered_negative_time_diffs).describe())

最大的间隔位于: 394528574
时间间隔大于6000的个数有：72
负先导 394000000.0-396500000.0 阶段的时间间隔分布情况：
                   0
count     72.000000
mean    8668.583333
std     2466.294830
min     6033.000000
25%     7003.000000
50%     7842.500000
75%     9893.000000
max    17017.000000


In [306]:
from sklearn.cluster import DBSCAN
from scipy import stats
# 计算梯级步长
## Step1: 拿到每个间隔对应的位置范围
filtered_indices = np.where(time_diffs > thea)
filtered_start_loc = negative_lead_result.Start_loc.iloc[filtered_indices[0]].values    # .values 将结果转为Numpy数组
start_locs = filtered_start_loc[:-1]  # 去掉最后一个元素
end_locs = filtered_start_loc[1:]    # 去掉第一个元素
# 拼接为元组列表
loc_tuples = list(zip(start_locs, end_locs))

# --- DBSCAN 聚类参数 ---
epsilon = 300  
min_samples = 3

distances = []
# Step2: 三维结果中筛选出对应的结果
for i, loc_tuple in enumerate(loc_tuples):
    # 按时间范围筛选结果
    filtered_loc_3d_result = filtered_3d_result[
        (filtered_3d_result.yld_start_loc > loc_tuple[0] - 1) &
        (filtered_3d_result.yld_start_loc < loc_tuple[1] + 1)
    ]
    if len(filtered_loc_3d_result) > min_samples:
        
        # ====================DBSCAN聚类去噪====================
        points_in_window = filtered_loc_3d_result[['x', 'y', 'z']].values
        db = DBSCAN(eps=epsilon, min_samples=min_samples).fit(points_in_window)
        cluster_labels = db.labels_
        core_samples_labels = cluster_labels[cluster_labels != -1]
        # 找到出现次数最多的簇ID
        main_cluster_id = stats.mode(core_samples_labels).mode
        main_path_mask = (cluster_labels == main_cluster_id)
        if np.sum(main_path_mask) < 2:
            continue
        # 使用掩码从DataFrame中获取主路径的所有数据
        main_path_df = filtered_loc_3d_result[main_path_mask]
        # 对筛选出的主路径点，按时间重新排序
        main_path_sorted_df = main_path_df.sort_values(by='yld_start_loc')
        # 获取并输出主路径的起止位置
        #    .iloc[0] 获取第一行, .iloc[-1] 获取最后一行
        start_position = main_path_sorted_df.iloc[0][['x','y','z']].values
        end_position = main_path_sorted_df.iloc[-1][['x','y','z']].values
        
# Step3: 计算三维距离（步长）
        distance = np.linalg.norm(end_position-start_position)
        distances.append(distance)
    

distances_df = pd.DataFrame(data=distances)
print(f"负先导 {start_loc}-{end_loc} 阶段的梯级步长分布情况：\n",distances_df.describe())
        

C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample 

负先导 394000000.0-396500000.0 阶段的梯级步长分布情况：
                 0
count   26.000000
mean   130.012783
std     80.194046
min     43.654846
25%     59.422754
50%    107.940293
75%    183.104362
max    336.238342


### 2500-4000 高度

In [311]:
# 选出负先导数据
start_loc = 3.965e8
end_loc = 3.98e8
negative_lead_result = filtered_result[
    (filtered_result.Start_loc >= start_loc) &
    (filtered_result.Start_loc < end_loc)
]
time_diffs = np.diff(negative_lead_result.Start_loc)
max_diff_idx = np.argmax(time_diffs)
corresponding_start_loc = negative_lead_result.Start_loc.iloc[max_diff_idx]
print("最大的间隔位于:",corresponding_start_loc)

# 查看时间间隔大于某阈值的个数

thea = 3000
filtered_negative_time_diffs = time_diffs[ time_diffs > thea]
print(f"时间间隔大于{thea}的个数有：{len(filtered_negative_time_diffs)}")
print(f"负先导 {start_loc}-{end_loc} 阶段的时间间隔分布情况：\n",pd.DataFrame(data=filtered_negative_time_diffs).describe())

最大的间隔位于: 396642080
时间间隔大于3000的个数有：68
负先导 396500000.0-398000000.0 阶段的时间间隔分布情况：
                  0
count    68.000000
mean   4323.220588
std    1197.745766
min    3009.000000
25%    3376.750000
50%    4058.000000
75%    4798.250000
max    8460.000000


In [312]:
from sklearn.cluster import DBSCAN
from scipy import stats
# 计算梯级步长
## Step1: 拿到每个间隔对应的位置范围
filtered_indices = np.where(time_diffs > thea)
filtered_start_loc = negative_lead_result.Start_loc.iloc[filtered_indices[0]].values    # .values 将结果转为Numpy数组
start_locs = filtered_start_loc[:-1]  # 去掉最后一个元素
end_locs = filtered_start_loc[1:]    # 去掉第一个元素
# 拼接为元组列表
loc_tuples = list(zip(start_locs, end_locs))

# --- DBSCAN 聚类参数 ---
epsilon = 200  
min_samples = 3

distances = []
# Step2: 三维结果中筛选出对应的结果
for i, loc_tuple in enumerate(loc_tuples):
    # 按时间范围筛选结果
    filtered_loc_3d_result = filtered_3d_result[
        (filtered_3d_result.yld_start_loc > loc_tuple[0] - 1) &
        (filtered_3d_result.yld_start_loc < loc_tuple[1] + 1)
    ]
    if len(filtered_loc_3d_result) > min_samples:
        
        # ====================DBSCAN聚类去噪====================
        points_in_window = filtered_loc_3d_result[['x', 'y', 'z']].values
        db = DBSCAN(eps=epsilon, min_samples=min_samples).fit(points_in_window)
        cluster_labels = db.labels_
        core_samples_labels = cluster_labels[cluster_labels != -1]
        # 找到出现次数最多的簇ID
        main_cluster_id = stats.mode(core_samples_labels).mode
        main_path_mask = (cluster_labels == main_cluster_id)
        if np.sum(main_path_mask) < 2:
            continue
        # 使用掩码从DataFrame中获取主路径的所有数据
        main_path_df = filtered_loc_3d_result[main_path_mask]
        # 对筛选出的主路径点，按时间重新排序
        main_path_sorted_df = main_path_df.sort_values(by='yld_start_loc')
        # 获取并输出主路径的起止位置
        #    .iloc[0] 获取第一行, .iloc[-1] 获取最后一行
        start_position = main_path_sorted_df.iloc[0][['x','y','z']].values
        end_position = main_path_sorted_df.iloc[-1][['x','y','z']].values
        
# Step3: 计算三维距离（步长）
        distance = np.linalg.norm(end_position-start_position)
        distances.append(distance)
    

distances_df = pd.DataFrame(data=distances)
print(f"负先导 {start_loc}-{end_loc} 阶段的梯级步长分布情况：\n",distances_df.describe())
        

C:\Users\19229\AppData\Local\Temp\ipykernel_33516\4229713236.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\4229713236.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\4229713236.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\4229713236.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample 

负先导 396500000.0-398000000.0 阶段的梯级步长分布情况：
                 0
count   28.000000
mean   144.056596
std     82.778563
min     11.287030
25%     81.586063
50%    142.964007
75%    203.259484
max    302.754497


### 1000-2500 高度

In [313]:
# 选出负先导数据
start_loc = 3.989e8
end_loc = 4e8
negative_lead_result = filtered_result[
    (filtered_result.Start_loc >= start_loc) &
    (filtered_result.Start_loc < end_loc)
]
time_diffs = np.diff(negative_lead_result.Start_loc)
max_diff_idx = np.argmax(time_diffs)
corresponding_start_loc = negative_lead_result.Start_loc.iloc[max_diff_idx]
print("最大的间隔位于:",corresponding_start_loc)

# 查看时间间隔大于某阈值的个数

thea = 2000
filtered_negative_time_diffs = time_diffs[ time_diffs > thea]
print(f"时间间隔大于{thea}的个数有：{len(filtered_negative_time_diffs)}")
print(f"负先导 {start_loc}-{end_loc} 阶段的时间间隔分布情况：\n",pd.DataFrame(data=filtered_negative_time_diffs).describe())

最大的间隔位于: 398939424
时间间隔大于2000的个数有：37
负先导 398900000.0-400000000.0 阶段的时间间隔分布情况：
                  0
count    37.000000
mean   2566.000000
std     438.713523
min    2014.000000
25%    2152.000000
50%    2568.000000
75%    2830.000000
max    3520.000000


In [315]:
from sklearn.cluster import DBSCAN
from scipy import stats
# 计算梯级步长
## Step1: 拿到每个间隔对应的位置范围
filtered_indices = np.where(time_diffs > thea)
filtered_start_loc = negative_lead_result.Start_loc.iloc[filtered_indices[0]].values    # .values 将结果转为Numpy数组
start_locs = filtered_start_loc[:-1]  # 去掉最后一个元素
end_locs = filtered_start_loc[1:]    # 去掉第一个元素
# 拼接为元组列表
loc_tuples = list(zip(start_locs, end_locs))

# --- DBSCAN 聚类参数 ---
epsilon = 300  
min_samples = 3

distances = []
# Step2: 三维结果中筛选出对应的结果
for i, loc_tuple in enumerate(loc_tuples):
    # 按时间范围筛选结果
    filtered_loc_3d_result = filtered_3d_result[
        (filtered_3d_result.yld_start_loc > loc_tuple[0] - 1) &
        (filtered_3d_result.yld_start_loc < loc_tuple[1] + 1)
    ]
    if len(filtered_loc_3d_result) > min_samples:
        
        # ====================DBSCAN聚类去噪====================
        points_in_window = filtered_loc_3d_result[['x', 'y', 'z']].values
        db = DBSCAN(eps=epsilon, min_samples=min_samples).fit(points_in_window)
        cluster_labels = db.labels_
        core_samples_labels = cluster_labels[cluster_labels != -1]
        # 找到出现次数最多的簇ID
        main_cluster_id = stats.mode(core_samples_labels).mode
        main_path_mask = (cluster_labels == main_cluster_id)
        if np.sum(main_path_mask) < 2:
            continue
        # 使用掩码从DataFrame中获取主路径的所有数据
        main_path_df = filtered_loc_3d_result[main_path_mask]
        # 对筛选出的主路径点，按时间重新排序
        main_path_sorted_df = main_path_df.sort_values(by='yld_start_loc')
        # 获取并输出主路径的起止位置
        #    .iloc[0] 获取第一行, .iloc[-1] 获取最后一行
        start_position = main_path_sorted_df.iloc[0][['x','y','z']].values
        end_position = main_path_sorted_df.iloc[-1][['x','y','z']].values
        
# Step3: 计算三维距离（步长）
        distance = np.linalg.norm(end_position-start_position)
        distances.append(distance)
    

distances_df = pd.DataFrame(data=distances)
print(f"负先导 {start_loc}-{end_loc} 阶段的梯级步长分布情况：\n",distances_df.describe())
        

负先导 398900000.0-400000000.0 阶段的梯级步长分布情况：
                 0
count   13.000000
mean   178.832180
std    120.979255
min     17.329304
25%     85.100499
50%    181.580345
75%    295.590608
max    355.157391


C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample size requirements.
  main_cluster_id = stats.mode(core_samples_labels).mode
C:\Users\19229\AppData\Local\Temp\ipykernel_33516\2279937337.py:32: SmallSampleWarning: One or more sample arguments is too small; all returned values will be NaN. See documentation for sample 

### 3.2 计算正先导的间隔

In [211]:
# 选出正先导数据
start_loc = 3.805e8
end_loc = 3.809e8
positive_lead_result = filtered_result[
    (filtered_result.Start_loc >= start_loc) &
    (filtered_result.Start_loc < end_loc)
]
time_diffs = np.diff(positive_lead_result.Start_loc)
max_diff_idx = np.argmax(time_diffs)
corresponding_start_loc = positive_lead_result.Start_loc.iloc[max_diff_idx]
print("最大的间隔位于:",corresponding_start_loc)

# 查看时间间隔大于某阈值的个数

thea = 7000
filtered_positive_time_diffs = time_diffs[ time_diffs > thea]
print(f"时间间隔大于{thea}的个数有：{len(filtered_positive_time_diffs)}")
print(f"正先导 {start_loc}-{end_loc} 阶段的时间间隔分布情况：\n",pd.DataFrame(data=filtered_positive_time_diffs).describe())

最大的间隔位于: 380827505
时间间隔大于7000的个数有：0
正先导 380500000.0-380900000.0 阶段的时间间隔分布情况：
          0
count  0.0
mean   NaN
std    NaN
min    NaN
25%    NaN
50%    NaN
75%    NaN
max    NaN


In [218]:
# 选出正先导数据
start_loc = 3.844e8
end_loc = 3.848e8
positive_lead_result = filtered_result[
    (filtered_result.Start_loc >= start_loc) &
    (filtered_result.Start_loc < end_loc)
]
time_diffs = np.diff(positive_lead_result.Start_loc)
max_diff_idx = np.argmax(time_diffs)
corresponding_start_loc = positive_lead_result.Start_loc.iloc[max_diff_idx]
print("最大的间隔位于:",corresponding_start_loc)

# 查看时间间隔大于某阈值的个数

thea = 7000
filtered_positive_time_diffs = time_diffs[ time_diffs > thea]
print(f"时间间隔大于{thea}的个数有：{len(filtered_positive_time_diffs)}")
print(f"正先导 {start_loc}-{end_loc} 阶段的时间间隔分布情况：\n",pd.DataFrame(data=filtered_positive_time_diffs).describe())

最大的间隔位于: 384740992
时间间隔大于7000的个数有：0
正先导 384400000.0-384800000.0 阶段的时间间隔分布情况：
          0
count  0.0
mean   NaN
std    NaN
min    NaN
25%    NaN
50%    NaN
75%    NaN
max    NaN


In [230]:
# 选出正先导数据
start_loc = 3.844e8
end_loc = 3.848e8
positive_lead_result = filtered_result[
    (filtered_result.Start_loc >= start_loc) &
    (filtered_result.Start_loc < end_loc)
]
time_diffs = np.diff(positive_lead_result.Start_loc)
max_diff_idx = np.argmax(time_diffs)
corresponding_start_loc = positive_lead_result.Start_loc.iloc[max_diff_idx]
print("最大的间隔位于:",corresponding_start_loc)

# 查看时间间隔大于某阈值的个数

thea = 7000
filtered_positive_time_diffs = time_diffs[ time_diffs > thea]
print(f"时间间隔大于{thea}的个数有：{len(filtered_positive_time_diffs)}")
print(f"正先导 {start_loc}-{end_loc} 阶段的时间间隔分布情况：\n",pd.DataFrame(data=filtered_positive_time_diffs).describe())

最大的间隔位于: 384740992
时间间隔大于7000的个数有：0
正先导 384400000.0-384800000.0 阶段的时间间隔分布情况：
          0
count  0.0
mean   NaN
std    NaN
min    NaN
25%    NaN
50%    NaN
75%    NaN
max    NaN


In [246]:
# 选出正先导数据
start_loc = 3.872e8
end_loc = 3.875e8
positive_lead_result = filtered_result[
    (filtered_result.Start_loc >= start_loc) &
    (filtered_result.Start_loc < end_loc)
]
time_diffs = np.diff(positive_lead_result.Start_loc)
max_diff_idx = np.argmax(time_diffs)
corresponding_start_loc = positive_lead_result.Start_loc.iloc[max_diff_idx]
print("最大的间隔位于:",corresponding_start_loc)

# 查看时间间隔大于某阈值的个数

thea = 7000
filtered_positive_time_diffs = time_diffs[ time_diffs > thea]
print(f"时间间隔大于{thea}的个数有：{len(filtered_positive_time_diffs)}")
print(f"正先导 {start_loc}-{end_loc} 阶段的时间间隔分布情况：\n",pd.DataFrame(data=filtered_positive_time_diffs).describe())

最大的间隔位于: 387219131
时间间隔大于7000的个数有：0
正先导 387200000.0-387500000.0 阶段的时间间隔分布情况：
          0
count  0.0
mean   NaN
std    NaN
min    NaN
25%    NaN
50%    NaN
75%    NaN
max    NaN
